In [2]:
import pandas as pd
from plots.utils import compute_user_penalty, get_team_values_df
from common.load import load_competition_data, process_team_logs

# Import common data

In [3]:
config = 'config_vbs2023.yaml'

# load competition data from dres files and auxiliary data (FPSs, sequences)
comp_data = load_competition_data(config)

# compute team logs and put them in the form of dataframes (or load them if already existing)
logs = process_team_logs(config, comp_data, force=False)

teams = comp_data['config']['teams']

/disks/workspace/Workspace/VBS-Analysis/VBS-2022/common/tasks.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.tasks_df = self.tasks_df.append({
/disks/workspace/Workspace/VBS-Analysis/VBS-2022/common/tasks.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.tasks_df = self.tasks_df.append({
/disks/workspace/Workspace/VBS-Analysis/VBS-2022/common/tasks.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.tasks_df = self.tasks_df.append({
/disks/workspace/Workspace/VBS-Analysis/VBS-2022/common/tasks.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.tasks_df = self.tasks_df.append({
/disks/workspace/Workspace/V

# Analysis

## Time Recall Table

In [27]:
def compute_data_for_time_recall_table(comp_data, teams, logs, split_user=False, only_best_user=False, max_records=10000):
    dfs = []
    for team in teams:
        team_df = logs[team].get_events_dataframe().reset_index()
        team_df = get_team_values_df(comp_data, team_df, split_user if not only_best_user else True, max_records)
        dfs.append(team_df)

    total_df = pd.concat(dfs, axis=0).reset_index()

    # keep the results from the best user among the two, for each given (task, team)
    if only_best_user:
        # the best penalty contribution is given by the ranks. If they are equal, then the user that submitted earlier wins
        user_penalty = compute_user_penalty(total_df, max_records)
                    
        total_df['user_penalty'] = user_penalty
        total_df = total_df.loc[total_df.groupby(['team', 'task'])['user_penalty'].idxmin()]
        total_df = total_df.drop(['user_penalty'], axis=1)
    
    return total_df

def build_table(df, teams, dataset='v3c'):
    # drop unuseful columns
    df = df.drop(['time_first_appearance', 'rank_shot_first_appearance', 'time_last_appearance', 'rank_shot_last_appearance', 'time_first_appearance_video', 'rank_video_first_appearance'], axis=1)

    #renaming task
    rename_fun = lambda x: x.replace('vbs23-kis-t', 'T_').replace('vbs23-kis-v-m', 'M_').replace('vbs23-kis-v', 'V_')
    df['task'] = df['task'].apply(rename_fun)
    tasks = df[['task','task_start']].sort_values(by=['task_start'])['task'].unique()
    textual=[t for t in tasks if t.startswith('T')] #used later to order the column
    visual=[t for t in tasks if t.startswith('V')] #used later to order the column
    marine=[t for t in tasks if t.startswith('M')] #used later to order the column
    df.drop(columns='task_start', inplace=True)

    df = df.fillna(-1)
    col=[c for c in df.columns.values.tolist() if c!='team' and c!='task' and c!='user']
    df[col]= df[col].astype('int32')
    df[col] = df[col].applymap(lambda x: -1 if x< 0 else x)
    df = df.astype('str')
    df.replace(['-1'], '-', inplace=True)

    #aggregate
    agg_dic={c: (lambda x: ' / '.join(x)) for c in col}
    agg_dic['time_correct_submission']="min"
    df=df.groupby(['team','task'])[col].agg(agg_dic).reset_index()
    df.replace('- / -', '-', regex=True, inplace=True)
    add_second= lambda x: x if x=='-' else x + 's'
    df['time_correct_submission']=df['time_correct_submission'].apply(add_second)
    df['time_best_shot']=df['time_best_shot'].apply(add_second)
    # df['rank_shot_margin_5'] = df['rank_shot_margin_5'].apply(add_second)
    df['time_best_video'] = df['time_best_video'].apply(add_second)

    # put ! where no logs are present
    tasks_unique = pd.DataFrame({'task': textual+visual+marine})
    teams_unique = pd.DataFrame({'team': df['team'].unique()})
    task_team = teams_unique.merge(tasks_unique, how='cross')
    df = task_team.merge(df, how='left').fillna('!')

    df = df.melt(var_name="metric", id_vars=["team", "task"], value_name="value")
    df['unit'] = df['metric'].apply(lambda x: 'rank' if x.startswith('rank_') else 'time')
    replace_dic = {
        'rank_shot_margin_0': 'correct frame',
        'time_best_shot': 'correct frame',
        'rank_shot_margin_5': 'frame in GT+2x5s',
        'time_best_shot_margin5': 'frame in GT+2x5s',
        'rank_video': 'correct video',
        'time_best_video': 'correct video',
        'time_correct_submission': 'correct submission'
    }
    df['metric']=df['metric'].map(replace_dic)
    df = df.pivot(index=['team', 'metric', 'unit'], columns="task", values="value")
    df=df.fillna('-')


    #sorting index desired order
    level_0=teams #order in the conf file
    level_1=['correct frame', 'frame in GT+2x5s', 'correct video','correct submission']
    level_2=['rank','time']
    df = df.reindex(pd.MultiIndex.from_product([level_0,level_1,level_2]))
    df.dropna(axis=0, inplace=True)#'correct submission'/rank shluld not be in the index
    df_margin_5 = df.copy()

    # sorting index desired order
    level_0 = teams  # order in teh conf file
    level_1 = ['correct frame', 'correct video', 'correct submission']
    level_2 = ['rank', 'time']
    df = df.reindex(pd.MultiIndex.from_product([level_0, level_1, level_2]))
    df.dropna(axis=0, inplace=True)  # 'correct submission'/rank shluld not be in the index

    #ordering columns
    if dataset == 'v3c':
        df = df[textual+visual]
    elif dataset == 'mvk':
        df = df[marine]
    else:
        # put all
        df = df[textual+visual+marine]
    return df, df_margin_5

In [28]:
df = compute_data_for_time_recall_table(comp_data, teams, logs)
table, table5 = build_table(df, teams, dataset='v3c')
table

task                                 T_1   T_2   T_3   T_4   T_5   T_6   T_7  \
VISIONE    correct frame      rank     1     1     1    16     1     1     4   
                              time   26s   65s  384s   76s   88s   23s   64s   
           correct video      rank     1     1     1    16     1     1     4   
                              time   26s   59s  384s   76s   88s   23s   64s   
           correct submission time   65s   76s     -  111s  100s   52s   95s   
VideoCLIP  correct frame      rank     1  1335     -     -     -    40     !   
                              time   51s  334s     -     -     -  175s     !   
           correct video      rank     1  1335    13    25   904    40     !   
                              time   51s  334s  191s  174s  371s  175s     !   
           correct submission time  283s     -     -  198s     -     -     !   
Verge      correct frame      rank     3    79   638   132    39     3  4453   
                              time   40s  160s  289s  345s  271s  131s   26s   
           correct video      rank     3    79   124   132    14     3    25   
                              time   40s  160s  210s  345s  369s  131s   27s   
           correct submission time  281s     -     -  356s     -  180s  115s   
HTW        correct frame      rank     1     1    12    13     4    16     -   
                              time   28s    6s  277s  140s  104s   21s     -   
           correct video      rank     1     1    12    13     1    16     1   
                              time   28s    6s  277s  140s   79s   21s   23s   
           correct submission time   37s   26s  395s  155s  150s   29s  193s   
VIREO      correct frame      rank     6   141   100    20   201     5     -   
                              time  137s  409s   53s  187s   50s   24s     -   
           correct video      rank     1   141    64    20     6     5     1   
                              time   20s  409s  280s  187s   91s   24s  176s   
           correct submission time  166s     -     -  253s  122s  311s  191s   
vitrivr    correct frame      rank     4     4     2   289     1    81    22   
                              time   23s   78s   32s   88s  110s   26s   39s   
           correct video      rank     2     4     2     7     1    81     1   
                              time   47s   78s   32s   88s  110s   26s   39s   
           correct submission time   94s  219s     -  115s  116s   63s   50s   
vitrivr-VR correct frame      rank     -     -     -     -     -     3     -   
                              time  124s  153s   40s   21s   53s   34s   39s   
           correct video      rank     -     -     3    63     -     2     1   
                              time  124s  153s   40s  395s   53s   34s   39s   
           correct submission time  213s     -   93s     -  179s   53s   88s   

task                                 V_1   V_2  V_3   V_4   V_5   V_6  
VISIONE    correct frame      rank     4  3752    2    24    19   224  
                              time   16s   46s  26s   18s   62s  298s  
           correct video      rank     1   106    1    24     1     2  
                              time   16s   87s  26s   18s   65s  163s  
           correct submission time   25s  112s  34s   25s   72s     -  
VideoCLIP  correct frame      rank     -     !    !     !     !     !  
                              time     -     !    !     !     !     !  
           correct video      rank    28     !    !     !     !     !  
                              time   48s     !    !     !     !     !  
           correct submission time   53s     !    !     !     !     !  
Verge      correct frame      rank   518     -  393    82   122     9  
                              time  201s     -  18s   55s   65s  204s  
           correct video      rank    17    86    2    82   122     2  
                              time  201s   33s  25s   55s   65s  120s  
           correct submission time  222

In [31]:
# exclude videoclip and VERGE that have not logged MVK
df_filtered = df[~df['team'].isin(['VideoCLIP', 'Verge'])]

table, table5 = build_table(df_filtered, teams, dataset='mvk')
table

task                                M_1   M_2   M_3   M_4   M_5  M_7
VISIONE    correct frame      rank    6    80    17    33    27    2
                              time  23s  205s   12s  245s   19s  37s
           correct video      rank    4    80    17    33    27    1
                              time  23s  205s   12s  245s   19s  12s
           correct submission time  76s  254s   27s  266s   51s  44s
HTW        correct frame      rank    8     4     6     5    41    3
                              time  16s   30s   15s   94s   23s  11s
           correct video      rank    8     1     3     5    41    1
                              time  16s   30s   15s   94s   23s  11s
           correct submission time  27s   46s   24s  211s   53s  23s
VIREO      correct frame      rank  132    78   216   188   198    5
                              time  16s   20s   16s  101s  219s   7s
           correct video      rank   26    78    40   188   198    4
                              time  16s   20s   72s  101s  219s   7s
           correct submission time  43s  110s  101s  183s     -  11s
vitrivr    correct frame      rank    -     -     -     -     -    -
                              time    -     -     -     -     -    -
           correct video      rank   46    24     6   641    16    1
                              time  21s  201s  179s  219s   38s  16s
           correct submission time  80s     -  196s     -   56s  22s
vitrivr-VR correct frame      rank  104     -    43     -   545    0
                              time  13s  165s  160s  286s   66s  16s
           correct video      rank   98     -    43   170   545    0
                              time  15s  165s  160s  215s   66s  16s
           correct submission time  38s  270s   62s  265s  102s  26s